In [12]:
import torch
import torch.nn as nn
from typing import Any

In [13]:
__all__ = ['Alexnet', 'alexnet']

model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}

In [15]:
class AlexNet(nn.Module):
  def __init__(self, num_classes: int=1000) -> None:
    super(AlexNet, self).__init__()
    self.features = nn.Sequential(
        # Conv1
        # Input channel:3, output channel:64, kernel size: 11, stride: 4, padding: 2
        nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
        nn.ReLU(inplace=True), # inplace 연산은 결과값을 새로운 변수에 값을 저장하는 대신 기존의 데이터를 대체하는 것을 의미
        # Max Pool1
        nn.MaxPool2d(kernel_size=3, stride=2),

        # Conv2
        nn.Conv2d(64, 192, kernel_size=5, padding=2),
        nn.ReLU(inplace=True),
        # Max Pool2
        nn.MaxPool2d(kernel_size=3, stride=2),

        # Conv3
        nn.Conv2d(192, 384, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        
        # Conv4
        nn.Conv2d(384, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),

        # Conv5
        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        # Max Pool3
        nn.MaxPool2d(kernel_size=3, stride=2),
    ) 
    self.avgpool = nn.AdaptiveAveragePool2d((6, 6))
    self.classifier = nn.Sequential(
      # 드롭아웃
      nn.Dropout(),
      nn.Linear(256 * 6 * 6, 4096),
      nn.ReLU(inplace=True),

      nn.Dropout(),
      nn.Linear(4096, 4096),
      nn.ReLU(inplace=True),

      nn.Linear(4096, num_classes),
  )
  def forward(self, x: torch.Tensor) -> torch.Tensor:
    # 특징 추출 부분
    x = self.features(x)

    x = self.avgpool(x)
    # output shape: (batch size * 256(channel), 6, 6)
    # Flatten
    x = torch.flatten(x, 1)
    # output shape (batch size, 256 * 6 * 6)
    # 분류 
    x = self.classifier(x)
    return x